# 📊 04 - Analyse de Données avec Spark
Ce notebook a pour objectif de fournir des données d'analyse à partir des données déjà transformées, nettoyées et enrichies.


In [1]:


# 0. Stoppe toute session existante
try:
    spark.stop()
except:
    pass

# 1. Recréation SparkSession avec 16 Go de RAM alloués
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .appName("04_analyse")
        .master("local[*]")  # Utilise tous les cœurs disponibles
        .config("spark.hadoop.fs.defaultFS", "file:///")
        .config("spark.driver.host", "127.0.0.1")
        .config("spark.driver.bindAddress", "0.0.0.0")
        .config("spark.driver.memory", "16g")         # <-- alloue 16 Go au driver Spark
        .config("spark.executor.memory", "16g")       # <-- alloue 16 Go aux tâches exécutées (optionnel en local)
        .config("spark.sql.shuffle.partitions", "8")  # <-- limite les partitions pour limiter le thread/mémoire
        .getOrCreate()
)


In [2]:
from pyspark.sql.functions import col, lower, trim, count
import os

# 1. Chargement
path = os.path.abspath(os.path.join(os.getcwd(), "../data/step3_enriched_csv"))
df_enriched = spark.read.option("header", "true").option("sep", ";").csv(path)

In [3]:
# Marques les plus fréquentes
top_brands = df_enriched.groupBy("brands") \
    .count() \
    .orderBy("count", ascending=False) \
    .filter(col("brands").isNotNull() & (col("brands") != "")) \
    .limit(30)

# Sauvegarde pour la visualisation
top_brands.coalesce(1) \
    .write.option("header", "true") \
    .mode("overwrite") \
    .csv("../data/viz/top_brands")

In [4]:
# Pays les plus fréquents
top_countries = df_enriched.groupBy("country") \
    .count() \
    .orderBy("count", ascending=False) \
    .filter(col("country").isNotNull() & (col("country") != "")) \
    .limit(20)

top_countries.coalesce(1) \
    .write.option("header", "true") \
    .mode("overwrite") \
    .csv("../data/viz/top_countries")

In [5]:
# 📊 Moyenne score_env_composite_flexible (impact environnementale) par pays (classés par fréquence d’apparition)
from pyspark.sql import functions as F

# Fréquence des pays
country_freq = df_enriched.groupBy("country").count()

# Moyenne des scores environnementaux
score_by_country = df_enriched.groupBy("country") \
    .agg(F.avg("score_env_composite_flexible").alias("avg_score_env_composite_flexible"))

# Jointure et tri par fréquence décroissante
result_country = score_by_country.join(country_freq, on="country") \
    .orderBy(F.desc("count"))

result_country.coalesce(1) \
    .write.option("header", "true") \
    .option("sep", ";") \
    .mode("overwrite") \
    .csv("../data/viz/score_env_by_country")


In [6]:
# Moyenne score_composite (aliments sains) par pays (classés par fréquence d’apparition) 

score_composite_by_country = df_enriched.groupBy("country") \
    .agg(F.avg("score_composite").alias("avg_score_composite"))

result_composite_country = score_composite_by_country.join(country_freq, on="country") \
    .orderBy(F.desc("count"))

result_composite_country.coalesce(1) \
    .write.option("header", "true") \
    .option("sep", ";") \
    .mode("overwrite") \
    .csv("../data/viz/score_composite_by_country")


In [7]:
# 🏷️ Moyenne score_env_composite_flexible par marque (classées par fréquence)

brand_freq = df_enriched.groupBy("brands").count()

score_by_brand = df_enriched.groupBy("brands") \
    .agg(F.avg("score_env_composite_flexible").alias("avg_score_env_composite_flexible"))

result_brand = score_by_brand.join(brand_freq, on="brands") \
    .orderBy(F.desc("count"))

result_brand.coalesce(1) \
    .write.option("header", "true") \
    .option("sep", ";") \
    .mode("overwrite") \
    .csv("../data/viz/score_env_by_brand")


In [8]:
# 🏷️ Moyenne score_composite par marque (classées par fréquence)

score_composite_by_brand = df_enriched.groupBy("brands") \
    .agg(F.avg("score_composite").alias("avg_score_composite"))

result_composite_brand = score_composite_by_brand.join(brand_freq, on="brands") \
    .orderBy(F.desc("count"))

result_composite_brand.coalesce(1) \
    .write.option("header", "true") \
    .option("sep", ";") \
    .mode("overwrite") \
    .csv("../data/viz/score_composite_by_brand")


In [9]:
from pyspark.sql.functions import col, avg, max as spark_max, when, lit
from pyspark.sql.types import BooleanType

# 🧱 1. Base des métriques quantitatives
metrics_cols = [
    col("score_env_composite_flexible").cast("double").alias("score_env_composite_flexible"),
    col("score_composite").cast("double").alias("score_composite"),
    col("energy-kcal_100g").cast("double").alias("energy_kcal_100g")
]

# 🔎 2. Colonnes booléennes personnalisées
bool_cols = [c for c in df_enriched.columns if c.startswith("is_")]
print(f"✅ Colonnes booléennes détectées ({len(bool_cols)}) :", bool_cols)

# 🧼 3. Forcer le cast vers booléen + nulls → False
df_clean = df_enriched
for c in bool_cols:
    df_clean = df_clean.withColumn(
        c,
        when(col(c).cast(BooleanType()).isNull(), lit(False)).otherwise(col(c).cast(BooleanType()))
    )

# 🧮 4. Création de la base avec cast vers int pour aggregation
df_base = df_clean.select(
    col("country"), col("brands"),
    *metrics_cols,
    *[col(c).cast("int").alias(c) for c in bool_cols]
)
df_base.createOrReplaceTempView("df_base")

# 📊 5. Génération dynamique de la requête CUBE
base_expr = """
    country,
    brands,
    AVG(score_env_composite_flexible) AS avg_env_score,
    AVG(score_composite) AS avg_composite_score,
    AVG(energy_kcal_100g) AS avg_kcal,
    GROUPING(country) AS grouping_country,
    GROUPING(brands) AS grouping_brands,
    GROUPING_ID(country, brands) AS grouping_id
"""
bool_exprs = ",\n    ".join([
    f"MAX({c}) = 1 AS has_{c.replace('is_', '')}" for c in bool_cols
])
query = f"""
SELECT
    {base_expr},
    {bool_exprs}
FROM df_base
GROUP BY CUBE(country, brands)
"""

df_final = spark.sql(query)

# ✅ Vérification
has_cols = [f"has_{c.replace('is_', '')}" for c in bool_cols]
null_counts = df_final.select([
    col(c).isNull().cast("int").alias(c + "_nulls") for c in has_cols
]).groupBy().sum().collect()[0].asDict()

print("📋 Vérification finale des nulls :")
for k, v in null_counts.items():
    print(f" - {k.replace('_nulls','')}: {v} ligne(s) nulles")

# Remplacement éventuel des derniers nulls
from pyspark.sql.functions import coalesce
for c in has_cols:
    df_final = df_final.withColumn(c, coalesce(col(c), lit(False)))

# 💾 Export final
df_final.cache()
print("✅ Nombre de lignes dans df_final :", df_final.count())

df_final.coalesce(1) \
    .write.option("header", "true") \
    .option("sep", ";") \
    .mode("overwrite") \
    .csv("../data/viz/cube_agg_country_brand_scores_enriched")


✅ Colonnes booléennes détectées (6) : ['is_vegan', 'is_vegetarian', 'is_sans_sucre', 'is_protein_plus', 'is_light', 'is_ultra_transformed']
📋 Vérification finale des nulls :
 - sum(has_vegan): 0 ligne(s) nulles
 - sum(has_vegetarian): 0 ligne(s) nulles
 - sum(has_sans_sucre): 0 ligne(s) nulles
 - sum(has_protein_plus): 0 ligne(s) nulles
 - sum(has_light): 0 ligne(s) nulles
 - sum(has_ultra_transformed): 0 ligne(s) nulles
✅ Nombre de lignes dans df_final : 684661
